<a href="https://colab.research.google.com/github/apotheosis-ai/MusicLSTM/blob/main/MusicLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importing Required Packages**


In [4]:
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import mido
import tensorflow as tf
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

##**Load MID file/files**

In [6]:
!unzip /content/chillhopdata.zip -d chillhop

Archive:  /content/chillhopdata.zip
  inflating: chillhop/1.mid          
  inflating: chillhop/10.mid         
  inflating: chillhop/11.mid         
  inflating: chillhop/12.mid         
  inflating: chillhop/13.mid         
  inflating: chillhop/14.mid         
  inflating: chillhop/15.mid         
  inflating: chillhop/16.mid         
  inflating: chillhop/17.mid         
  inflating: chillhop/18.mid         
  inflating: chillhop/19.mid         
  inflating: chillhop/2.mid          
  inflating: chillhop/20.mid         
  inflating: chillhop/3.mid          
  inflating: chillhop/4.mid          
  inflating: chillhop/5.mid          
  inflating: chillhop/6.mid          
  inflating: chillhop/7.mid          
  inflating: chillhop/8.mid          
  inflating: chillhop/9.mid          
  inflating: chillhop/Cymatics - Eternity MIDI 1 - C Maj.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 10 - F Min.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 11 - A Maj.mid  
  inflating: 

In [ ]:
import os

notes = []
for song in os.listdir("/content/chillhop"):
  mid = MidiFile('/content/chillhop/' + song)
  print(mid)

  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
      data = msg.bytes()
      notes.append(data[1])

##**Scale Data**

In [11]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1,1)))

In [12]:
notes

[array([0.35483871]),
 array([0.48387097]),
 array([0.38709677]),
 array([0.15053763]),
 array([0.31182796]),
 array([0.27956989]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.48387097]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.48387097]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.59139785]),
 array([0.55913978]),
 array([0.53763441]),
 array([0.51612903]),
 array([0.55913978]),
 array([0.30107527]),
 array([0.09677419]),
 array([0.22580645]),
 array([0.33333333]),
 array([0.25806452]),
 array([0.15053763]),
 array([0.22580645]),
 array([0.25806452]),
 array([0.31182796]),
 array([0.33333333]),
 array([0.61290323]),
 array([0.38709677]),
 array([0.48387097]),
 array([0.35483871]),
 array([0.51612903]),
 array([0.69892473]),
 array([0.68817204]),
 array([0.56989247]),
 array([0.59139785]),
 array([0.07526882]),
 array([0.15053763]),
 array([0.1827957]),
 array([0.23655914]),
 array([0.2

##**Create Train Data**

##**Build LSTM**

##**Training**

##**Generating & Saving LSTM Music**

In [ ]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')